In [ ]:
pip install -q torch torchvision segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from segmentation_models_pytorch import MAnet, Linknet, FPN, PSPNet, PAN
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from glob import glob

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
num_epochs = 120
learning_rate = 0.001

In [ ]:
device

device(type='cuda')

In [ ]:
class CottonDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images = sorted(glob(os.path.join(images_dir, "*.jpg")))
        self.masks = sorted(glob(os.path.join(masks_dir, "*.jpg")))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("RGB")
        mask = Image.open(self.masks[idx]).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = (mask > 0).float()  # Convert mask to binary
        return image, mask

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [ ]:
!unzip /content/drive/MyDrive/data.zip -d /content/final_data

Archive:  /content/drive/MyDrive/data.zip
   creating: /content/final_data/data/
  inflating: /content/final_data/data/.DS_Store  
   creating: /content/final_data/data/images/
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1004.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1008.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1011.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1013.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1018.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1022.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1023.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1030.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_mosaic1034.jpg  
  inflating: /content/final_data/data/images/20221109_lbk_p4r_twri_

In [ ]:
images_dir = "/content/final_data/data/images"
masks_dir = "/content/final_data/data/masks"

In [ ]:
image_files = glob(os.path.join(images_dir, "*.jpg"))
print(f"Number of images in images_dir: {len(image_files)}")

mask_files = glob(os.path.join(masks_dir, "*.jpg"))
print(f"Number of images in masks_dir: {len(mask_files)}")

Number of images in images_dir: 1866
Number of images in masks_dir: 1866


In [ ]:
dataset = CottonDataset(images_dir, masks_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# model = MAnet(encoder_name="resnet34", encoder_weights="imagenet", classes=1, activation=None)
# model = model.to(device)

In [ ]:
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0
#     for images, masks in train_loader:
#         images, masks = images.to(device), masks.to(device)

#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs, masks)
#         train_loss += loss.item()

#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     avg_train_loss = train_loss / len(train_loader)
#     print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")

In [ ]:
def calculate_iou(pred_mask, true_mask):
    pred_mask = (pred_mask > 0.5).float()  # Thresholding
    intersection = (pred_mask * true_mask).sum()
    union = (pred_mask + true_mask).sum() - intersection
    iou = intersection / union if union != 0 else 1.0
    return iou.item()

In [ ]:
# model.eval()
# iou_scores = []
# with torch.no_grad():
#     for images, masks in val_loader:
#         images, masks = images.to(device), masks.to(device)

#         # Predict
#         preds = model(images)
#         preds = torch.sigmoid(preds)  # Apply sigmoid for binary classification

#         # Calculate IoU for each image in the batch
#         for pred, mask in zip(preds, masks):
#             iou = calculate_iou(pred, mask)
#             iou_scores.append(iou)

# mean_iou = np.mean(iou_scores)
# print(f"Mean IoU Score on Validation Set: {mean_iou:.4f}")

In [ ]:
# Configuration
encoder_name = "resnet34"  # You can experiment with different encoders here
encoder_weights = "imagenet"
in_channels = 3
classes = 1
# activation = "sigmoid"

# Initialize models
models = {
    "Manet" :  MAnet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes, activation=None),
    "LinkNet": Linknet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes, activation=None),
    "FPN": FPN(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes, activation=None),
    "PSPNet": PSPNet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes, activation=None),
    "PAN": PAN(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes, activation=None),
}

# Move models to device
for model_name, model in models.items():
    models[model_name] = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 176MB/s]


In [ ]:
# Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam

# Train and Evaluate Each Model
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")

    # Optimizer for the model
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            train_loss += loss.item()

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], {model_name} Train Loss: {avg_train_loss:.4f}")

    # Evaluation - IoU Calculation
    iou_scores = []
    model.eval()
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            preds = model(images)
            preds = torch.sigmoid(preds)  # Apply sigmoid for binary classification

            # Calculate IoU for each image in the batch
            for pred, mask in zip(preds, masks):
                iou = calculate_iou(pred, mask)
                iou_scores.append(iou)

    mean_iou = np.mean(iou_scores)
    print(f"{model_name} Mean IoU Score on Validation Set: {mean_iou:.4f}\n")

Training and evaluating Manet...
Epoch [1/120], Manet Train Loss: 0.2184
Epoch [2/120], Manet Train Loss: 0.1450
Epoch [3/120], Manet Train Loss: 0.1403
Epoch [4/120], Manet Train Loss: 0.1363
Epoch [5/120], Manet Train Loss: 0.1335
Epoch [6/120], Manet Train Loss: 0.1265
Epoch [7/120], Manet Train Loss: 0.1186
Epoch [8/120], Manet Train Loss: 0.1130
Epoch [9/120], Manet Train Loss: 0.1082
Epoch [10/120], Manet Train Loss: 0.1070
Epoch [11/120], Manet Train Loss: 0.1031
Epoch [12/120], Manet Train Loss: 0.1150
Epoch [13/120], Manet Train Loss: 0.1130
Epoch [14/120], Manet Train Loss: 0.1048
Epoch [15/120], Manet Train Loss: 0.0998
Epoch [16/120], Manet Train Loss: 0.0965
Epoch [17/120], Manet Train Loss: 0.0943
Epoch [18/120], Manet Train Loss: 0.0932
Epoch [19/120], Manet Train Loss: 0.0943
Epoch [20/120], Manet Train Loss: 0.0899
Epoch [21/120], Manet Train Loss: 0.0883
Epoch [22/120], Manet Train Loss: 0.0869
Epoch [23/120], Manet Train Loss: 0.0862
Epoch [24/120], Manet Train Loss:

### IOU scores
##### 1) Manet Mean IoU Score on Test Set: 0.5797
##### 2) LinkNet Mean IoU Score on Test Set: 0.5769
##### 3) FPN Mean IoU Score on Test Set: 0.5710
##### 4) PSPNet Mean IoU Score on Test Set: 0.5478
##### 5) PAN Mean IoU Score on Validation Set: 0.5498


